# People Analytics: Análise de Desempenho e Retenção

Objetivo: conduzir uma análise prática usando people_data.csv para (1) ler os dados, (2) calcular métricas-chave (média salarial, tempo de casa, turnover), (3) visualizar distribuição por departamento e tempo de empresa, e (4) interpretar achados e propor recomendações de RH para retenção e performance.

In [ ]:
# (2) Leitura de dados
import pandas as pd
import numpy as np

# Ajuste o path se necessário
df = pd.read_csv('../data/people_data.csv')
print(df.shape)
df.head()

In [ ]:
# (3) Métricas agregadas: média salarial, tempo de casa, turnover
from datetime import datetime

# Supondo colunas: 'employee_id', 'department', 'salary', 'hire_date', 'termination_date' (NaN se ativo), 'performance_score'
df['hire_date'] = pd.to_datetime(df['hire_date'], errors='coerce')
df['termination_date'] = pd.to_datetime(df.get('termination_date'), errors='coerce')
ref_date = pd.to_datetime(df.get('snapshot_date'), errors='coerce')
ref_date = ref_date.iloc[0] if ref_date.notna().any() else pd.Timestamp.today().normalize()

# Tempo de casa em anos
end_date = df['termination_date'].fillna(ref_date)
df['tenure_years'] = (end_date - df['hire_date']).dt.days / 365.25

# Ativo x desligado
df['active'] = df['termination_date'].isna()

# Métricas globais
avg_salary = df['salary'].mean()
avg_tenure = df['tenure_years'].mean()
turnover_rate = 1 - df['active'].mean()  # proporção desligada

print({'avg_salary': round(avg_salary,2), 'avg_tenure_years': round(avg_tenure,2), 'turnover_rate': round(turnover_rate,3)})

# Métricas por departamento
dept_metrics = df.groupby('department').agg(
    headcount=('employee_id', 'nunique'),
    avg_salary=('salary', 'mean'),
    avg_tenure=('tenure_years', 'mean'),
    turnover_rate=('active', lambda s: 1 - s.mean()),
    avg_perf=('performance_score','mean')
).reset_index()
dept_metrics['avg_salary'] = dept_metrics['avg_salary'].round(2)
dept_metrics['avg_tenure'] = dept_metrics['avg_tenure'].round(2)
dept_metrics['turnover_rate'] = dept_metrics['turnover_rate'].round(3)
dept_metrics['avg_perf'] = dept_metrics['avg_perf'].round(2)
dept_metrics.sort_values('turnover_rate', ascending=False)

In [ ]:
# (4) Visualizações: distribuição por departamento e tempo de empresa
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style='whitegrid')

fig, axes = plt.subplots(1, 2, figsize=(14,5))

# Distribuição de headcount por departamento
dept_counts = df['department'].value_counts().sort_values(ascending=False)
sns.barplot(x=dept_counts.values, y=dept_counts.index, ax=axes[0], color='#4C72B0')
axes[0].set_title('Headcount por Departamento')
axes[0].set_xlabel('Colaboradores')
axes[0].set_ylabel('Departamento')

# Distribuição de tempo de casa
sns.histplot(df['tenure_years'].dropna(), bins=20, kde=True, ax=axes[1], color='#55A868')
axes[1].set_title('Distribuição de Tempo de Casa (anos)')
axes[1].set_xlabel('Anos de empresa')
axes[1].set_ylabel('Colaboradores')
plt.tight_layout()
plt.show()

# Boxplot salário por departamento (extra)
plt.figure(figsize=(12,5))
sns.boxplot(data=df, x='department', y='salary')
plt.title('Salário por Departamento')
plt.xticks(rotation=30, ha='right')
plt.tight_layout()
plt.show()

## Interpretação e Recomendações

- Se o turnover estiver elevado em certos departamentos, revisar práticas de liderança, cargas de trabalho e trilhas de carreira.
- Programas de reconhecimento e ajuste de faixas salariais podem reduzir evasão se salários e performance não estiverem alinhados.
- Reforçar onboarding e mentoria para grupos com menor tempo de casa e maior rotatividade.
- Monitorar coortes por ano de contratação para entender riscos de saída antecipada.
- Estabelecer metas de OKRs por time e ciclos de feedback contínuo para elevar performance.

Próximos passos: modelar propensão ao desligamento (churn) usando variáveis de satisfação, remuneração, senioridade e gestor; testar intervenções A/B em políticas de retenção.